<a href="https://colab.research.google.com/github/Czedros/CSE352-Machine-Learning-Assignments/blob/main/PrototypeHeatmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
import folium
from folium.plugins import HeatMap

nybb_url = "https://raw.githubusercontent.com/dwillis/nyc-maps/master/boroughs.geojson"
nyc = gpd.read_file(nybb_url).to_crs(epsg=4326)
nyc_land = nyc.unary_union



# These points approximate real trip-dense zones
hotspot_centers = [
    ("Midtown",           40.7549, -73.9840),  # Times Sq / Penn Station
    ("Lower Manhattan",   40.7128, -74.0060),  # FiDi
    ("Harlem",            40.8121, -73.9465),  # Harlem hospitals
    ("Downtown Brooklyn", 40.6925, -73.9904),  # Court St / MetroTech
    ("Flushing",          40.7580, -73.8300),  # Main St
    ("Jamaica",           40.7027, -73.8000),  # LIRR / E train hub
    ("Fordham",           40.8623, -73.8970),  # Bronx shopping corridor
    ("St George",         40.6430, -74.0736)   # Staten Island ferry
]

def generate_cluster(lat, lon, n_points, spread=0.01):
    """Generate clustered points around a hotspot center."""
    pts = []
    for _ in range(n_points):
        new_lat = np.random.normal(lat, spread)
        new_lon = np.random.normal(lon, spread)
        p = Point(new_lon, new_lat)
        if nyc_land.contains(p):   # make sure it's land only
            pts.append((new_lat, new_lon))
    return pts



heat_data = []

# Strong hotspots (visually obvious)
for name, lat, lon in hotspot_centers:
    pts = generate_cluster(lat, lon, n_points=200, spread=0.005)
    heat_data.extend(pts)

# Softer secondary clusters
for name, lat, lon in hotspot_centers:
    pts = generate_cluster(lat, lon, n_points=100, spread=0.01)
    heat_data.extend(pts)

# Light background noise (non-hotspot but still valid)
def random_point_in_polygon(poly):
    minx, miny, maxx, maxy = poly.bounds
    while True:
        p = Point(np.random.uniform(minx, maxx),
                  np.random.uniform(miny, maxy))
        if poly.contains(p):
            return (p.y, p.x)

for _ in range(300):
    heat_data.append(random_point_in_polygon(nyc_land))


m = folium.Map(location=[40.73, -73.97], zoom_start=11, tiles="CartoDB Positron")

HeatMap(
    heat_data,
    radius=18,
    blur=25,
    min_opacity=0.4
).add_to(m)

m.save("nyc_heatmap_hotspots.html")

m


/tmp/ipython-input-1715776869.py:12: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  nyc_land = nyc.unary_union
